In [1]:
#cifar 10 dataset
import torch
import torchvision
import torchvision.transforms as transforms
import os
import numpy as np
from tqdm import tqdm
import json
"""
File format:
<pred label> <true label>
101001000011111....
"""

N: int = 50000
PATH:str = 'trainset_cifar10_small.onnx'
ONLY_CORRECT_PRED = True #whether we should extract pattern from wrong predictions or not


all_patterns = {}

for idx in tqdm(range(N)):
    with open(os.path.join(PATH, "reluBitmap_{}.txt").format(idx)) as f:
        raw = f.readlines()
        pred_label, true_label = raw[0].strip().split()
        if ONLY_CORRECT_PRED:
            if pred_label != true_label: continue
        bitmap = [*raw[1].strip()] #convert the string "01000..." to the list ["0", "1", "0", "0", ...]
        bitmap = [int(b) for b in bitmap] #convert the list of string to the list of int
        if pred_label in all_patterns:
            all_patterns[pred_label].append(bitmap)
        else:
            all_patterns[pred_label] = []
            all_patterns[pred_label].append(bitmap)

for k in all_patterns.keys():
    all_patterns[k] = np.array(all_patterns[k])

    print(k, all_patterns[k].shape)



100%|██████████| 50000/50000 [00:22<00:00, 2270.11it/s]


6 (3539, 2568)
9 (3891, 2568)
1 (4079, 2568)
2 (2372, 2568)
7 (3518, 2568)
8 (3999, 2568)
4 (3075, 2568)
3 (2477, 2568)
0 (3344, 2568)
5 (3291, 2568)


In [5]:
DELTA = 0.05
stable_relu_patterns = {}
for k in all_patterns.keys():
    print(k)
    sum = all_patterns[k].sum(axis = 0)
    n_sample = all_patterns[k].shape[0]
    print("n_sample", n_sample)
    neg_relus = np.where(sum <= n_sample*DELTA)
    pos_relus = np.where(sum >= n_sample*(1-DELTA))
    stable_idx = np.concatenate([neg_relus, pos_relus], axis= 1).squeeze()
    print(stable_idx)
    stable_idx = np.sort(stable_idx)
    
    stable_relu_patterns[k] = {"stable_idx": stable_idx.tolist(), 
                               "val": [int(sum[idx]) for idx in stable_idx]}

with open("datasets/CIFAR10/cifar10_relu_patterns_{}.json", "w") as f:
    json.dump(stable_relu_patterns, f, indent=2)


6
n_sample 3539
[2379 2383 2384 2386 2394 2397 2401 2406 2409 2410 2411 2422 2426 2439
 2441 2446 2448 2452 2455 2456 2457 2462 2467 2469 2474 2478 2485 2486
 2489 2493 2498 2500 2501 2520   66   67   68   69   81   82   83   84
   85   97   98   99  100  112  113  114  129 2514 2524 2525 2526 2528
 2529 2535 2540 2541 2548 2551 2553 2559 2562 2564]
9
n_sample 3891
[2382 2383 2385 2388 2393 2394 2395 2399 2402 2407 2412 2415 2416 2419
 2420 2423 2424 2425 2427 2428 2429 2430 2435 2436 2440 2441 2445 2448
 2454 2456 2458 2461 2463 2473 2477 2479 2480 2482 2483 2487 2490 2494
 2495 2497 2530 2534 2537 2538 2550 2552 2504 2505 2507 2510 2514 2518
 2519 2520 2532 2539 2546 2547 2549 2554 2557]
1
n_sample 4079
[2376 2382 2383 2389 2393 2394 2398 2399 2404 2406 2407 2410 2415 2418
 2419 2420 2424 2427 2428 2429 2430 2435 2436 2438 2443 2445 2448 2449
 2456 2458 2461 2463 2477 2479 2480 2483 2487 2491 2494 2495 2497 2522
 2526 2530 2534 2537 2538 2550 2552 2560 2505 2507 2510 2532 2539 2544
 

In [ ]:
"""
Check accuracy of the loaded model. Expected: 74.16%
"""
from sklearn.metrics import confusion_matrix, accuracy_score
true_labels = []
pred_labels = []
for idx in range(len(test_images)):
    input = (test_images[idx]/255).astype(np.float32).reshape(1,32,32,3)
    # print(input, input.shape)
    pred = pytorch_model(torch.Tensor(input))
    # print(pred, np.argmax(pred), labels[idx])
    true_labels.append(test_labels[idx])
    if pred is not None:
        pred_labels.append(torch.argmax(pred))
    else:
        pred_labels.append(-1)
print(confusion_matrix(true_labels, pred_labels))
print(accuracy_score(true_labels, pred_labels))

[[775  16  33  21  18   4   6  10  67  50]
 [ 12 833   3  10   3   2  11   2  24 100]
 [ 57   4 560  85 122  75  51  26   6  14]
 [ 10   5  47 563  66 212  42  22  14  19]
 [ 19   3  34  67 734  33  29  64  12   5]
 [ 14   2  29 154  55 687  12  37   5   5]
 [  7   5  24  77  50  28 796   3   4   6]
 [ 10   1  19  45  60  74   5 773   2  11]
 [ 34  31   7  18   4   3   3   5 863  32]
 [ 24  64   6  20   7   6   3  12  26 832]]
0.7416


In [ ]:
idx = 0
input = (test_images[idx]/255).astype(np.float32).reshape(1,32,32,3)
print(marabou_network.evaluate(input))
print(pytorch_model(torch.Tensor(input)))
marabou_network.reluList

In [ ]:
print(len(test_images))

10000
